# Examen de la práctica 2 de APR, grupo 4CO21, 10 de enero de 2024
# Turno 1: de 19h a 19.45h

## El conjunto de datos caltech-101

Caltech-101 consta de imágenes de objetos de 101 clases, más una clase "cajón de sastre". Cada imagen está etiquetada con un único objeto. Cada clase contiene entre 40 y 800 imágenes aproximadamente, con alrededor de 9.000 imágenes en total. Las imágenes son de tamaños variables, de anchos y altos típicamente entre 200 y 300 píxeles. A continuación se carga este conjunto de datos mediante la librería tensorflow_datasets y se divide en una parte para entrenamiento (train_data), otra para validación (val_data) y otra para test (test_data).

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
train_data, val_data, test_data = tfds.load('caltech101', split=['train[:80%]', 'train[80%:]', 'test'], as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3060 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/caltech101/3.0.2.incomplete1WUT76/caltech101-train.tfrecord*...:   0%|    …

Generating test examples...:   0%|          | 0/6084 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/caltech101/3.0.2.incomplete1WUT76/caltech101-test.tfrecord*...:   0%|     …

Dataset caltech101 downloaded and prepared to /root/tensorflow_datasets/caltech101/3.0.2. Subsequent calls will reuse this data.


## Ejercicio

Como hicimos en las sesiones 3 y 4 de la práctica, haz un fine-tuning (ajuste fino) de una red Keras pre-entrenada con ImageNet que clasifique el test de caltech-101 con la máxima precisión posible, preferiblemente superior al 80%.

In [2]:
from keras.applications.efficientnet import preprocess_input

img_size = (300, 200)
num_classes = 102

def preprocess(image, label):
    image = tf.image.resize(image, img_size)
    image = tf.cast(image, tf.float32)
    image = preprocess_input(image)
    label = tf.one_hot(label, num_classes)
    return image, label

In [3]:
train_data = train_data.map(preprocess)
test_data = test_data.map(preprocess)
val_data = val_data.map(preprocess)

In [4]:
#train_size = int(0.8 * train_size)
train_dataset = train_data
val_dataset = val_data
test_dataset = test_data

print(len(train_dataset),len(val_dataset))

2448 612


In [5]:
from keras.applications.efficientnet_v2 import EfficientNetV2S

model = EfficientNetV2S(input_shape=img_size + (3,), include_top=False, weights='imagenet')
#model.summary()

82420632/82420632 [==============================] - 0s 0us/step


In [6]:
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model

for layer in model.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=model.input, outputs=output)

#model.summary()

In [7]:
from keras.optimizers import Adam

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

In [8]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=5
batch_size=32
train_dataset_batched = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset_batched = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
history = model.fit(train_dataset_batched,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_dataset_batched,
                    callbacks=[reduce_lr,checkpoint])

Epoch 1/5
77/77 [==============================] - ETA: 0s - loss: 1.9878 - accuracy: 0.6352
Epoch 1: val_accuracy improved from -inf to 0.87092, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


77/77 [==============================] - 47s 368ms/step - loss: 1.9878 - accuracy: 0.6352 - val_loss: 0.4636 - val_accuracy: 0.8709 - lr: 0.0010
Epoch 2/5
77/77 [==============================] - ETA: 0s - loss: 0.3261 - accuracy: 0.9134
Epoch 2: val_accuracy improved from 0.87092 to 0.89052, saving model to best_model.h5
77/77 [==============================] - 18s 239ms/step - loss: 0.3261 - accuracy: 0.9134 - val_loss: 0.3477 - val_accuracy: 0.8905 - lr: 0.0010
Epoch 3/5
77/77 [==============================] - ETA: 0s - loss: 0.1707 - accuracy: 0.9559
Epoch 3: val_accuracy improved from 0.89052 to 0.89869, saving model to best_model.h5
77/77 [==============================] - 16s 208ms/step - loss: 0.1707 - accuracy: 0.9559 - val_loss: 0.3314 - val_accuracy: 0.8987 - lr: 0.0010
Epoch 4/5
77/77 [==============================] - ETA: 0s - loss: 0.1074 - accuracy: 0.9726
Epoch 4: val_accuracy improved from 0.89869 to 0.90033, saving model to best_model.h5
77/77 [=====================

In [9]:
model = load_model('best_model.h5')
test_dataset_batched = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
score = model.evaluate(test_dataset_batched, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Test loss: 43.52
Test accuracy: 86.93
